In [3]:
from datasets import  load_dataset

dataset = load_dataset("ranWang/UN_PDF_TEXT_DATA", split="randomTest")

Found cached dataset parquet (C:/Users/wRan/.cache/huggingface/datasets/ranWang___parquet/ranWang--UN_PDF_TEXT_DATA-8444a7274238e7fd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [4]:
dataset

Dataset({
    features: ['zh', 'ar', 'en', 'fr', 'es', 'ru', 'record'],
    num_rows: 2839
})

In [21]:
def get_all_directory_contents(article_content):
    table_contents_line = []
    
    page_list = article_content.split("\n----\n")
    
    last_directory_page = 0
    for index, page in enumerate(page_list):
        for line in page.split("\n"):
            if "....." in line:
                table_contents_line.append(line)
                last_directory_page = index + 1
    
    if len(table_contents_line) < 4:
        return None, None
    
    return list(map(lambda s:s[:s.find("...")].replace(" ",""), table_contents_line)), last_directory_page

In [33]:
def preprocessin_file_content(file_content:str, last_directory_page:int):
    """
    预处理文件的内容，删除每一页中页脚和页眉，以及删除目录页之前的内容
    """
    page_list = file_content.split("\n----\n")[last_directory_page:]

    remove_footer_and_header_page = []
    
    for page in page_list:
        line_list = page.split("\n")
        last_index = len(line_list) - 1
        for index, line in enumerate(line_list):
            if "__________" in line:
                last_index = index
                break
        
        remove_footer_and_header_page.append("\n".join(line_list[3:last_index]))
        
        
    return "\n".join(remove_footer_and_header_page)

In [7]:
def find_last_sequence(index, paragraph_list):
    next_index = None
    for paragraph in paragraph_list[index+1:]:
        if re.search(r"^\d+\.", paragraph):
            next_index = paragraph.split(".")[0]
            break
            
    return next_index

In [8]:
def spilt_paragraph(current_index, next_index, line):
    if not line:
        return []
    
    slilt_list = []
    while True:   
        current_index = current_index + 1
        
        if current_index >= int(next_index):
            slilt_list.append(line)
            break 
      
        split_line_by_index = line.split(f"{current_index}.")
        if len(split_line_by_index) == 1:
            zh_split_line_by_index = line.split(f"{current_index}．")
            if len(zh_split_line_by_index) == 2:
                split_line_by_index = zh_split_line_by_index
                
        if len(split_line_by_index) == 1:
            slilt_list.append(split_line_by_index[0])
            break
            
        slilt_list.append(split_line_by_index[0])    
        line = f"{current_index}.{split_line_by_index[1]}"
        
    return slilt_list

In [9]:
def post_processing(paragraph_list, lang):
    processed_paragraph_list = []
    
    if lang == "zh":
        for index, paragraph in enumerate(paragraph_list):
            end_index = paragraph.rfind("。")
     
            if end_index < len(paragraph) - 6:
                processed_paragraph_list.append(paragraph[:end_index+1])
                
            else:
                processed_paragraph_list.append(paragraph)
                
    return processed_paragraph_list

In [45]:
import re

def process(row, directory, last_page):

    removed_directory_content_list = []
    for line in preprocessin_file_content(row, last_page).split("\n"):
        line_copy = line.replace(" ","")
        if line_copy in directory:
            continue

        removed_directory_content_list.append(line)


    imperfect_paragraph_list = []
    per_insert_paragraph = ""
    for paragraph in removed_directory_content_list:
        if re.search(r"^\d+\.", paragraph):
            imperfect_paragraph_list.append(per_insert_paragraph)
            per_insert_paragraph = paragraph
        else:
            per_insert_paragraph += paragraph
            
            
    paragraph_list = []
    last_index = 0

    for index, paragraph in enumerate(imperfect_paragraph_list):
        if not paragraph:
            continue

        match = re.search(r"^\d+\.", paragraph)
        if match:
            current_index = int(match[0].split(".")[0])
            if current_index < last_index:
                paragraph_list[-1] = paragraph_list[-1] + paragraph
                continue
                
            last_index = current_index
            next_index = find_last_sequence(index, paragraph_list)

            if next_index:
                for split_paragraph in spilt_paragraph(current_index, next_index, paragraph):
                    paragraph_list.append(split_paragraph)      
            else:
                paragraph_list.append(paragraph)
        else:
            if len(paragraph_list) > 0:
                paragraph_list[-1] = paragraph_list[-1] + paragraph
            
    return paragraph_list

In [46]:
directory, last_page = get_all_directory_contents(dataset[1]["en"])
paragraph_list = process(dataset[1]["en"], directory, last_page)
paragraph_list

['1. Bermuda is a Non-Self-Governing Territory administered by the United Kingdom of Great Britain and Northern Ireland.1 The Territory is located in the western part of the Atlantic Ocean, a bout 917 kilometres east of the nearest continental shore, the North Carolina coast of the United States of America. The Territory, covering 57 square kilometres, consists of 8 major and 130 smaller islands. The largest is Great Island, or Main Island. Hamilton, the capital, and St. George’s are the two main towns. ',
 '2. Discovered by Spanish navigator Juan de Bermúdez in 1505, Bermuda celebrated its 500th anniversary in 2005.2 In 1609, Bermuda was settled by a group of English colonists and in 1610 the first slaves were brought to work on the farms and to dive for pearls. In 1620, the first Parliament was held after the Crown granted the colony limited self-government. In 1968, Bermuda was granted internal self-government and a new Constitution. In 1995, a referendum was held on the question of

In [39]:
last_page

In [49]:
from pathlib import Path


def main(dataset):
    for row in dataset:
        directory, last_page = get_all_directory_contents(row["zh"])
        if directory:
            paragraph_list = process(row["zh"], directory, last_page)
            if paragraph_list:
                Path(f"./txt/{row['record']}").mkdir(parents=True, exist_ok=True)
                with open(f"./txt/{row['record']}/zh.txt", "w", encoding='utf-8') as f:
                    f.write("\n".join(paragraph_list))

In [50]:
main(dataset)